In [1]:
num_eps = [6,22,24,14,26,25,25,24,23]
episodes = []
for season in range(1,10):
    for ep in range(1,num_eps[season-1]+1):
        if ep < 10:
            ep_str = "s0" + str(season) + "e0" + str(ep)
        else:
            ep_str = "s0" + str(season) + "e" + str(ep)
        with open(ep_str, "r") as file:
            episodes.append(file.readline())

In [2]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

In [3]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(episodes)

In [4]:
df = pd.DataFrame(X.toarray())

In [7]:
vectorizer

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [20]:
for i in vectorizer.get_feature_names():
    if 't' == i:
        print(i)

In [5]:
df.shape

(189, 19238)

In [19]:
df = df.div(df.sum(axis=1), axis=0)

In [22]:
df.to_csv("bow.tsv", sep='\t', header=False, index=False)